# 2-buses UCBlock, NetworkBlock and two ThermalUnitBlocks
In the following, we provide an example on how to build a simple optimization model with SMS++. This is a dispatch problem, considering two buses, connected by a line, with a load and a thermal generator each. 

First, an empty SMSNetwork with block file type is generated.

In [ ]:
from pysmspp import SMSNetwork, SMSFileType

sn = SMSNetwork(file_type=SMSFileType.eBlockFile)  # Empty Block

sn

The network does not contain any block inside, so it has to be populated. The first block is a UCBlock, which includes several information.
1- Dimensions of the problem

In [ ]:
kwargs = {
    "TimeHorizon": 24,  # number of time steps
    "NumberUnits": 2,  # number of units
    "NumberElectricalGenerators": 2,  # number of electrical generators
    "NumberNodes": 2,  # number of nodes
    "NumberLines": 1,  # number of lines
}

2- Demand for each node. This has to be defined as a Variable object

In [ ]:
from pysmspp import Variable
import numpy as np

demand_array = np.full((2, 24), 50.0)
demand = {
    "ActivePowerDemand": Variable(  # active power demand
        "ActivePowerDemand",
        "float",
        ("NumberNodes", "TimeHorizon"),
        demand_array,
    )
}  # constant demand of 50kW

kwargs = {**kwargs, **demand}

3- Parameters for the line
In fact, a line can be described with a DCNetworkBlock or directly including its relevant parameters in UCBlock

In [ ]:
line_variables = {
    "StartLine": Variable("StartLine", "int", ("NumberLines",), [0]),
    "EndLine": Variable("EndLine", "int", ("NumberLines",), [1]),
    "MinPowerFlow": Variable("MinPowerFlow", "float", ("NumberLines",), [-50.0]),
    "MaxPowerFlow": Variable("MaxPowerFlow", "float", ("NumberLines",), [50.0]),
    "LineSusceptance": Variable("LineSusceptance", "float", ("NumberLines",), [0.0]),
}

kwargs = {**kwargs, **line_variables}

4- Variable to specify in which bus (node) the generator is attached

In [ ]:
generator_node = {
    "GeneratorNode": Variable(
        "GeneratorNode", int, ("NumberElectricalGenerators",), [0, 1]
    ),
}

kwargs = {**kwargs, **generator_node}

Add everything with the SMSnetwork.add function

In [ ]:
sn.add(
    "UCBlock",  # block type
    "Block_0",  # block name
    id="0",  # block id
    **kwargs,
)

sn

Now the SMSNetworj object has an UCBlock called Block_0. Let's see how it is organized

In [ ]:
sn.blocks["Block_0"]

Now, the two thermal units have to be added to the UCBlock as ThermalUnitBlocks.
First, the Block().from_kwargs is used to organize the object as a block

In [ ]:
from pysmspp import Block, SMSConfig

thermal_unit_block = Block().from_kwargs(
    block_type="ThermalUnitBlock",
    MinPower=Variable("MinPower", "float", (), 0.0),
    MaxPower=Variable("MaxPower", "float", (), 70.0),
    LinearTerm=Variable("LinearTerm", "float", (), 0.3),
    InitUpDownTime=Variable("InitUpDownTime", "int", (), 1),
)

thermal_unit_block

Then, the unit block is added to the UCBlock

In [ ]:
# Add it to the existing UCBlock (Block_0)
sn.blocks["Block_0"].add_block("UnitBlock_0", block=thermal_unit_block)

sn.blocks["Block_0"]

Similarly for the second ThermalUnitBlock. The max power is chosen to force the system to use the line

In [ ]:
thermal_unit_block = Block().from_kwargs(
    block_type="ThermalUnitBlock",
    MinPower=Variable("MinPower", "float", (), 0.0),
    MaxPower=Variable("MaxPower", "float", (), 90.0),
    LinearTerm=Variable("LinearTerm", "float", (), 0.8),
    InitUpDownTime=Variable("InitUpDownTime", "int", (), 1),
)

# Add it to the existing UCBlock (Block_0)
sn.blocks["Block_0"].add_block("UnitBlock_1", block=thermal_unit_block)

sn.blocks["Block_0"]

The problem can be now optimized

In [ ]:
configfile = SMSConfig(
    template="uc_solverconfig"
)  # path to the template solver config file "uc_solverconfig"
temporary_smspp_file = "./2buses_2thermal.nc"  # path to temporary SMS++ file
output_file = "./2buses_2thermal.txt"  # path to the output file (optional)

result = sn.optimize(
    configfile,
    temporary_smspp_file,
    output_file,
)

The value of the objective function and the complete log can be obtained with

In [ ]:
result.objective_value

In [ ]:
result.log